In [110]:
import os
import json
import torch
import os
import json

torch.cuda.is_available()
def json_standard(input_object):
    """transfrom python object into json and str with json format 从而增强gpt的成功率
    json.dumps(): 将 Python 对象编码成 JSON 字符串。
    json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。
    json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。
    json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
    """
    # print(input_object,type(input_object))
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist


In [111]:
# 计算message 和string中的token数量
import tiktoken
# 2. Load an encoding 使用tiktoken.get_encoding() 按名称加载编码。第一次运行时，它将需要互联网连接进行下载。后续运行不需要互联网连接。
encoding = tiktoken.get_encoding("cl100k_base")
# 使用tiktoken.encoding_for_model()函数可以自动加载给定模型名称的正确编码。
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# 3. Turn text into tokens with encoding.encode() The .encode() method converts a text string into a list of token integers.
encoding.encode("tiktoken is great!")
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """返回文本字符串中的Token数量"""
    num_tokens=0
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens
def calculate_messages(messages:list) -> int:
    """返回总的message长度"""
    counter=0
    for mess in messages:
        counter+=num_tokens_from_string(mess.content,"cl100k_base")
    return counter
INCONTEXT_WINDOWS_GPT_3_5=16000
INCONTEXT_WINDOWS_MOONSHOT=30000
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI
import os
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI
import time
# moonshot_chat.openai_api_base="https://api.moonshot.cn/v1"
# moonshot_chat=ChatOpenAI(temperature=0,model="moonshot-v1-32k",openai_api_key='XXX',openai_api_base='https://api.moonshot.cn/v1')
# moonshot_chat.temperature
class ChatGPTAnnotator:
    def __init__(self):
        self.chat = ChatOpenAI(temperature=0,
                               model="gpt-3.5-turbo",openai_api_key='XXX',
                               openai_api_base='https://gptnb.keqichen.top/v1')
    def init_judge_samecase_message(self):
        fewshot_0_0={"case a":"Therapist: In what way?\nClient: They think I’m ugly, hideous. I look like the elephant woman. They probably make jokes about me and have a good laugh. I can’t bear it.\nTherapist: Are those the thoughts that make you so anxious when you go into the high street?\nClient:  Yes.\n","case b":"Therapist:  One of your underlying assumptions is ‘If I don’t have a real job then I’m not as good as others’. Now you’ve been out of work for several years due to illness and you said you try to cram a lot of activity into each day to prove that you are as good as others by having a job at home, so to speak; so do you see reading the newspaper or watching television as an indulgence, idling when you should be striving?\nClient: Yes, that’s exactly it now that you’ve mentioned it. It is an indulgence. I should be working hard to make up for not having a real job. I don’t deserve any time to myself. I feel a failure if I don’t complete my daily to-do lists. \n"}
        fewshot_0_1={"reasoning":"No, case a is about appearance, while case b is related to unemployment. They belong to different counseling topics.","result":False}
        fewshot_1_0={
        "case a": "Therapist:That was quite a sigh. How are you feeling at this moment?\nClient:Down.\nTherapist:What thoughts are going through your mind right now to make you feel down?\nClient:I’ve let so many things slip through my fingers. What a waste. Why didn’t I grasp these opportunities and make the most of them?",
        "case b": "Therapist:What’s your answer to your own question?\nClient:I don’t have the guts to succeed. I’ll never amount to anything important in life. My life is an endless catalogue of missed opportunities."
        }
        fewshot_1_1={
        "reasoning": "Both cases is about a topic(missed opportunities) and the dialogues in these two cases are cohesive.",
        "result": True
        }
        messages=[SystemMessage(
            content="As a helpful AI and a professional psychotherapist. You need to answer question of each User question. Do case a and case b belong to the same counseling topic?. Return True and False. Please return a JSON format response with \"reasoning\" and \"result\". "
        ),
        HumanMessage(content=json_standard(fewshot_0_0)),
        AIMessage(content=json_standard(fewshot_0_1)),
        HumanMessage(content=json_standard(fewshot_1_0)),
        AIMessage(content=json_standard(fewshot_1_1))
        ]
        token_count=calculate_messages(messages)
        return messages,token_count

    def init_judge_qa_message(self):
        fewshot_0_0={"dialog a":"Client: I’d have to say “3”.","dialog b":"Therapist: So, what are the differences between “0” and “3”?"}
        fewshot_0_1={"reasoning":"These two dialog can form a dialog_pair because they represent an exchange between a client expressing a choice and a therapist seeking clarification","result":True}
        fewshot_1_0={"dialog a":"Client: I will have my things put away and I’ll be standing at my workstation.","dialog b":"Therapist: What did you come to see me about?"}
        fewshot_1_1={"reasoning":"They do not form a dialog pair. One is a statement from a client, while the other is a question from a therapist at the beginning of another session. There's no direct response or interaction to constitute a complete dialogue.","result":False}
        fewshot_2_0={"dialog a":"Client: Yes, that’s exactly it now that you’ve mentioned it. It is an indulgence. I should be working hard to make up for not having a real job. I don’t deserve any time to myself. I feel a failure if I don’t complete my daily to-do lists.","dialog b":"Therapist: That was quite a sigh. How are you feeling at this moment?"}
        fewshot_2_1={"reasoning":"Client in dialog a expresses guilt over leisure versus work, while Therapist acknowledges client's emotions and inquires about their current state. This continuity shows coherence and emotional connection. And the pronouns like 'that' refer to client's indication: 'It is an indulgence'.","result":True}
        fewshot_3_0={"dialog a":"Client: I’d have to say “3”.","dialog b":"Therapist: So, what are the differences between “0” and “3”?"}
        fewshot_3_1={"reasoning":"These two dialog can form a dialog_pair because they represent an exchange between a client expressing a choice and a therapist seeking clarification","result":True}
        messages = [
        SystemMessage(
            content="As a helpful AI and a professional psychotherapist, please help me determine whether dialog a and dialog b can be continue part of a multi-turn dialogue. Return true if they form a responsive dialogue pair, and false if they do not. Please return a JSON format response with \"reasoning\" and \"result\". "
        ),

        HumanMessage(content=json_standard(fewshot_2_0)),
        AIMessage(content=json_standard(fewshot_2_1)),
        HumanMessage(content=json_standard(fewshot_0_0)),
        AIMessage(content=json_standard(fewshot_0_1)),
        HumanMessage(content=json_standard(fewshot_1_0)),
        AIMessage(content=json_standard(fewshot_1_1))
        ]
        token_count=calculate_messages(messages)
        return messages,token_count
    # def case_judge_message(self):
    #     messages
    def retry_chat(self,messages,max_try_times:int=3):
        response=self.chat(messages)
        final_response_content=response.content
        counter=0
        response_metadata={"finish_reason": "stop", "logprobs": None}
        while "length" in response.response_metadata["finish_reason"] and counter<max_try_times:
            # print(len(messages),response)
            messages.append(AIMessage(
                content=final_response_content
            ))
            messages.append(HumanMessage(
                content="Well done! Please go on."
            ))
            response=self.chat(messages)
            final_response_content+=response.content
            counter+=1
            time.sleep(2)
        if counter==max_try_times:
            print(response.response_metadata)
            raise MyException("The response is too long! Max_try!!!!!!!")
        time.sleep(1)
        return final_response_content,response.response_metadata
    def judge_qa_message(self,input_json:str):
        """注意，传进来的值一定要经过json标准化"""
        json.loads(input_json)
        messages,token_count=self.init_judge_qa_message()
        messages.append(HumanMessage(
            content=input_json 
        ))      
        resp,meta_data=self.retry_chat(messages,3)
        return resp,meta_data
    def judge_case_message(self,input_json:str):
        messages,token_count=self.init_judge_samecase_message()
        messages.append(HumanMessage(
            content=input_json 
        ))
        resp,meta_data=self.retry_chat(messages,3)
        return resp,meta_data

In [112]:
a="1dd"
a.replace("1","4")

'4dd'

In [113]:
a={"1":"2","3":"4"}
json.dumps(a)
json

<module 'json' from '/home/ckqsudo/sys_tool/miniconda/tmp/yes/envs/trl/lib/python3.9/json/__init__.py'>

In [114]:
def a():
    return 1,3
b,c=a()

In [115]:
# from curses import raw


def judge_same_case(pre_clip:list,cur_clip:list,is_same_method,chat_item):
    if is_same_method==False:
        return 0,""
    # 这两个是同属一个case吗
    input_object={"case a":r"\n".join(pre_clip)+r"\n","case b":r"\n".join(cur_clip)+r"\n"}
    response,meta_data=chat_item.judge_case_message(json_standard(input_object))
    try:
        res_j=json.loads(response)
        result=1 if res_j["result"] else 0
        return result,res_j["reasoning"]
    except Exception as e: 
        print(meta_data,response)
        return None,None
        time.sleep(4)

def judge_qa(pre,cur,is_same_method,is_same_participant,chat_item):
    if is_same_method==False or is_same_participant==True:
        return 0,""
    # 这两个是同属一个QA吗
    is_qa=0
    input_object={"dialog a":pre,"dialog b":cur}
    response,meta_data=chat_item.judge_qa_message(json_standard(input_object))
    try:
        res_j=json.loads(response)
        result=1 if res_j["result"] else 0
        return result,res_j["reasoning"]
    except Exception as e: 
        print(meta_data,res_j)
        return None,None

        
chat_item=ChatGPTAnnotator()
def annotate_case(old_topic,cur_topic):
    pre_last_4=[item["participant"]+":"+item["text"] for item in old_topic["topic_dialog"]["dialog"][-4:]]
    cur_top_4=[item["participant"]+":"+item["text"] for item in cur_topic["topic_dialog"]["dialog"][:4]]
    is_same_method=old_topic["topic_dialog"]["method"]==cur_topic["topic_dialog"]["method"]
    is_same_participant=old_topic["topic_dialog"]["dialog"][-1]["participant"]==cur_topic["topic_dialog"]["dialog"][0]["participant"]
    is_same_qa,qa_reasoning=judge_qa(pre_last_4[-1],cur_top_4[0],is_same_method,is_same_participant,chat_item)
    is_same_case,case_reason=judge_same_case(pre_last_4,cur_top_4,is_same_method,chat_item)
    return is_same_qa,qa_reasoning,is_same_case,case_reason
def main():
    raw_input_dir="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/修正dialog输出文件夹"
    raw_output_dir="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/添加标记case文件夹"
    file_list=sorted(os.listdir(raw_input_dir),key=lambda x:int(x.split("_")[0]))
    for index,file_name in enumerate(file_list[:]):
        cur_file_path=raw_input_dir+"/"+file_name
        output_file_path=raw_output_dir+"/"+file_name
        if check_file(output_file_path)==False:
            cur_json=read_file(cur_file_path)
            if index==0:
                cur_json["is_same_qa"]=0
                cur_json["is_same_case"]=0
                write_file(output_file_path,cur_json)
            else:#index>0
                pre_file_path=raw_input_dir+"/"+file_list[index-1]
                pre_json=read_file(pre_file_path)
                is_same_qa,qa_reasoning,is_same_case,case_reasoning=annotate_case(pre_json,cur_json)
                if is_same_qa!=None and is_same_case!=None:
                    cur_json["is_same_qa"]=int(is_same_qa)
                    cur_json["is_same_case"]=int(is_same_case)
                    cur_json["extra_info"]=[qa_reasoning,case_reasoning]
                    write_file(output_file_path,cur_json)

In [116]:
main()

0_cache.json exist
1_cache.json exist
2_cache.json exist
3_cache.json exist
4_cache.json exist
5_cache.json exist
6_cache.json exist
7_cache.json exist
8_cache.json exist
9_cache.json exist
10_cache.json exist
11_cache.json exist
12_cache.json exist
13_cache.json exist
14_cache.json exist
15_cache.json exist
16_cache.json exist
17_cache.json exist
18_cache.json exist
19_cache.json exist
20_cache.json exist
21_cache.json exist
22_cache.json exist
23_cache.json exist
24_cache.json exist
25_cache.json exist
26_cache.json exist
27_cache.json exist
28_cache.json exist
29_cache.json exist
30_cache.json exist
31_cache.json exist
32_cache.json exist
33_cache.json exist
34_cache.json exist
35_cache.json exist
36_cache.json exist
37_cache.json exist
38_cache.json exist
39_cache.json exist
40_cache.json exist
41_cache.json exist
42_cache.json exist
43_cache.json exist
44_cache.json exist
45_cache.json exist
46_cache.json exist
47_cache.json exist
48_cache.json exist
49_cache.json exist
50_cache.j